In [98]:
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED)

In [99]:
chromosome_types = {
    'n_estimators': int,        # Number of trees (5 to 200)
    'max_depth': int,           # Max depth (10 to 100)
    'min_samples_split': int,   # Minimum samples to split (2 to 10)
    'min_samples_leaf': int,    # Minimum samples per leaf (1 to 10)
    'criterion': bool           # Criterion: 0:'gini' or 1:'entropy'
}

### Fitness function

In [100]:
def evaluate_fitness(chromosome, X_train, y_train, X_val, y_val):
    # Create and train the model
    model = RandomForestClassifier(**chromosome, random_state=SEED)
    model.fit(X_train, y_train)
    
    # Evaluate the model on the validation set
    predictions = model.predict(X_val)
    
    # Return the f1 score
    return balanced_accuracy_score(y_val, predictions)

### Population initialization

In [101]:
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        chromosome = {
            'n_estimators': random.randint(5, 200),
            'max_depth': random.randint(10, 100),
            'min_samples_split': random.randint(2, 10),
            'min_samples_leaf': random.randint(1, 10),
            'criterion': random.choice(["gini", "entropy"])
        }
        population.append(chromosome)
    return population

### Mutation operator

- For binary variables we just flip the bit with a probability $\tilde{p} = 1/n$ where $n$ is the size of the population.
- For positive integer variables we add a random number between $-1$ and $1$ to the variable.
- For real variables we add a random noise from a gaussian distribution with mean $0$ and standard deviation $0.1$.

In [102]:
def mutate(chromosome, pop_size):
    for gene in chromosome:
        gene_type = chromosome_types.get(gene, None)
        if gene_type == bool:
            chromosome[gene] = "gini" if chromosome[gene] == "entropy" else "entropy"
        elif gene_type == int:
            noise = random.randint(-1, 1)
            chromosome[gene] += noise
        elif gene_type == float:
            noise = random.normalvariate(0, 0.1)
            chromosome[gene] += noise
        else:
            raise ValueError(f"Unknown gene type")

### Crossover operator

In [103]:
def crossover(chromosome1, chromosome2):
    child = {}
    for gene in chromosome1:
        child[gene] = random.choice([chromosome1[gene], chromosome2[gene]])
    return child

### Selection operator

In [104]:
def select(population, X_train, y_train, X_val, y_val, tournament_size):
    # Tournament selection
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(population, tournament_size)
        winner = max(tournament, key=lambda c: evaluate_fitness(c, X_train, y_train, X_val, y_val))
        selected.append(winner)
    return selected

### Termination condition

In [105]:
def termination_condition(best_fitness, max_generations):
    return best_fitness == 1.0 or max_generations == 0

### Genetic algorithm

In [106]:
def genetic_algorithm(X_train, y_train, X_val, y_val, pop_size=100, max_generations=100, tournament_size=3, mutation_rate=0.1):
    # Initialize the population
    population = initialize_population(pop_size)
    
    # Evolution loop
    best_chromosome = None
    best_fitness = 0
    best_fitness_history = []
    for generation in tqdm(range(max_generations)):
        # Evaluate the population
        fitness = [evaluate_fitness(chromosome, X_train, y_train, X_val, y_val) for chromosome in population]
        
        # Select the best chromosome
        best_idx = fitness.index(max(fitness))
        if fitness[best_idx] > best_fitness:
            best_fitness = fitness[best_idx]
            best_chromosome = population[best_idx]
        
        # Termination condition
        best_fitness_history.append(best_fitness)
        if termination_condition(best_fitness, max_generations - generation):
            break
        
        # Select the next generation
        selected = select(population, X_train, y_train, X_val, y_val, tournament_size)
        
        # Crossover
        children = []
        for _ in range(pop_size // 2):
            parent1 = random.choice(selected)
            parent2 = random.choice(selected)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent1, parent2)
            children.extend([child1, child2])
        
        # Mutation
        for child in children:
            if random.random() < mutation_rate:
                mutate(child, pop_size)
        
        # Update the population
        population = children
    
    return best_chromosome, best_fitness, best_fitness_history

## Experiment

### Data loading

In [107]:
import pandas as pd

df = pd.read_parquet('datasets/creditcard.parquet')
df_train, df_val = train_test_split(df, test_size=0.2, random_state=SEED, stratify=df['Class'])

# Standardize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(df_train.drop(columns=['Class']))
y_train = df_train['Class'].values
X_val = scaler.transform(df_val.drop(columns=['Class']))
y_val = df_val['Class'].values

In [ ]:
# Run the genetic algorithm
best_chromosome, best_fitness, best_fitness_history = genetic_algorithm(
    X_train,
    y_train,
    X_val,
    y_val,
    pop_size=5,
    max_generations=20,
    tournament_size=3,
    mutation_rate=0.1
)

In [ ]:
import matplotlib.pyplot as plt

x_generations = list(range(1,1+len(best_fitness_history)))
plt.scatter(x_generations, best_fitness_history)
plt.plot(x_generations, best_fitness_history)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("Fitness through generations")
plt.show()